### Cosine Similarity 

In [2]:
pip install sentence-transformers sys

ERROR: Could not find a version that satisfies the requirement sys (from versions: none)
ERROR: No matching distribution found for sys
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import os
import sys
from sentence_transformers import SentenceTransformer, util
import re

sys.path.append(os.getcwd())

model = SentenceTransformer('stsb-roberta-large')

input_file_path = os.path.join('ARXIV', 'merged_data_with_references.json')
with open(input_file_path, 'r') as file:
    data = json.load(file)

output_data = []

for item in data:
    arxiv_id = item['arxivId']
    abstract = item['abstract']
    references = item['references']
    
    reference_similarities = []
    
    if abstract is not None:
        source_embedding = model.encode([abstract], convert_to_tensor=True)
        
        for ref in references:
            ref_id = ref['arxivId']
            ref_abstract = ref['abstract']
            
            if ref_abstract is not None:
                ref_embedding = model.encode([ref_abstract], convert_to_tensor=True)
                similarity = util.pytorch_cos_sim(source_embedding, ref_embedding).item()
                reference_similarities.append({'arxivId': ref_id, 'similarity': similarity})
    
    output_item = {
        'arxivId': arxiv_id,
        'references': reference_similarities
    }
    output_data.append(output_item)

output_file_path = os.path.join('ARXIV', 'similarity_data.json')
with open(output_file_path, 'w') as file:
    json.dump(output_data, file, indent=2)

print(f"Similarity data saved to {output_file_path}")

Similarity data saved to ARXIV/similarity_data.json
